In [ ]:
!git clone https://github.com/mariaaoprea/Diffusion-Models-for-floor-plan-drafting.git

In [ ]:
%cd Diffusion-Models-for-floor-plan-drafting
!pip install -r requirements.txt

In [ ]:
from diffusers import StableDiffusionPipeline
pipeline = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5"
).to("cuda")
pipeline.load_lora_weights("maria26/Floor_Plan_LoRA")

In [ ]:
!pip install Flask flask_cors pyngrok supabase

In [ ]:
import io
import os
import json
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from pyngrok import ngrok
from google.colab import userdata
from PIL import Image
from supabase import create_client, Client
import uuid
import datetime

app = Flask(__name__)

SUPABASE_URL = userdata.get('SUPABASE_URL')
SUPABASE_KEY = userdata.get('SUPABASE_KEY')
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

BUCKET_NAME = "chat-images"

CORS(
    app,
    resources={
        r"/api/*": {
            "origins": ["http://localhost:4200"],
            "supports_credentials": True,
            "allow_headers": ["Content-Type", "Authorization", "Accept"],
            "methods": ["GET", "POST", "OPTIONS"],
            "expose_headers": ["Content-Type"],
        }
    },
    send_wildcard=False
)

@app.route('/api/generate_image', methods=['POST'])
def generate_image():
    try:
        data = request.get_json()
        if not data or 'prompt' not in data:
            return jsonify({"error": "Missing 'prompt' in request."}), 400

        prompt = data['prompt']
        chat_id = data['chat_id']

        image = pipeline(prompt, num_inference_steps=50).images[0]

        img_io = io.BytesIO()
        image.save(img_io, format='PNG')
        img_io.seek(0)

        file_name = f"{chat_id}/{uuid.uuid4()}.png"

        supabase.storage.from_(BUCKET_NAME).upload(
            path=file_name,
            file=img_io.getvalue(),
            file_options={"content-type": "image/png"}
        )
        public_url = f"{SUPABASE_URL}/storage/v1/object/public/{BUCKET_NAME}/{file_name}"

        supabase.table("messages").insert({
            "chat_id": chat_id,
            "sender": "User",
            "message": prompt,
            "created_at": datetime.datetime.now(datetime.UTC).isoformat(),
            "image_url": None
        }).execute()


        supabase.table("messages").insert({
            "chat_id": chat_id,
            "sender": "AI",
            "message": prompt,
            "created_at": datetime.datetime.now(datetime.UTC).isoformat(),
            "image_url": public_url
        }).execute()


        return jsonify({
            "status": "success",
            "image_url": public_url
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/api/health', methods=['GET'])
def health_check():
    """Health check endpoint."""
    return jsonify({
        "status": "healthy",
        "version": "1.0.0"
    })


def start_server():
    ngrok.set_auth_token(userdata.get('NGROK_AUTH_TOKEN'))
    public_url = ngrok.connect(5000)
    print(f"API URL: {public_url.public_url}/api/generate_image")
    print(f"API URL: {public_url.public_url}/api/health")

    app.run(port=5000, threaded=True)

if __name__ == '__main__':
    start_server()